In [ ]:
from urllib.request import Request, urlopen
import pandas as pd
import urllib.error
import numpy as np
from bs4 import BeautifulSoup
import os
import time
import datetime
import re
import winsound

def gettext(url):
    hdr = {'User-Agent': 'Mozilla'}
    req = Request(url,headers=hdr) 
    try:
        with urlopen(req) as content:
            soup=BeautifulSoup(content)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(" ")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split(" :: "))
        text = ':'.join(chunk for chunk in chunks if chunk)
        text=text.replace("\xa0","")
        return text
    except (urllib.error.HTTPError,urllib.error.URLError):
        return "No text"


def dedup(adrslist):
    x=0
    y=1
    while y<len(adrslist):
        str1=adrslist[x]
        str2=adrslist[y]
        if str1[:10]==str2[:10]:
            adrslist.pop(x)
        else:
            x+=1
            y+=1
    return adrslist

def findadrs1(text,strtsufx,stateabrv):
    adrs1=list()
    for x in strtsufx:
        regex1A=r'(?<![^:|^\s]{1})[1-9]{1}\d{0,4}\s+[A-z0-9]{1}[^:]{0,25}'+x+'(?![A-z]{1})'
        adrs1A=re.findall(regex1A,text,re.IGNORECASE)
        if adrs1A:
            adrs1.append(adrs1A)
    for x in stateabrv:
        regex1B=r'(?<![^:|^\s]{1})[1-9]{1}\d{0,4}\s+'+x+'[0-9|\-]{1,3}'
        adrs1B=re.findall(regex1B,text,re.IGNORECASE)
        if adrs1B:
            adrs1.append(adrs1B)
    
    adrs1= [item for sublist in adrs1 for item in sublist]
    adrs1.sort()
    adrs1=dedup(adrs1)
    

    return adrs1

def findadrs2(text,adrs1,stateabrv):
    adrs2=list()
    for x in adrs1:
        for y in stateabrv:
            regex2A=r''+x+'.{0,35}'+y+'(?![A-z]{1})'
            regex2B=r''+x+'.{0,35}[0-9]{5}'
            adrs2A=re.findall(regex2A,text,re.IGNORECASE)
            adrs2B=re.findall(regex2B,text)
            if adrs2A:
                adrs2.append(adrs2A)
            if adrs2B:
                adrs2.append(adrs2B)
    adrs2= [item for sublist in adrs2 for item in sublist]
    adrs2.sort()
    adrs2=dedup(adrs2)
    return adrs2

def findadrs3(text,adrs1,stateabrv):
    adrs3=list()
    for x in adrs1:
        regex3=x
        text=re.sub(regex3,"&&&",text)
    for y in stateabrv:
        regex3A=r'(?<![^:|^\s]{1})[1-9]{1}\d{0,4}\s+[A-Z]{1}.{0,50}[0-9]{5}'
        regex3B=r'(?<![^:|^\s]{1})[1-9]{1}\d{0,4}\s+[A-Z]{1}.{0,50}'+y
        adrs3A=re.findall(regex3A,text,re.IGNORECASE)
        adrs3B=re.findall(regex3B,text)
        if adrs3A:
            adrs3.append(adrs3A)
        if adrs3B:
            adrs3.append(adrs3B)
    adrs3= [item for sublist in adrs3 for item in sublist]
    adrs3.sort()
    adrs3=dedup(adrs3)
    return adrs3

currenttime=str(datetime.datetime.now().strftime("%Y%m%d_%H%M"))
path='C:\\Users\\Isaac Choi\\Desktop\\_Input\\'
listofwebsitesdata=pd.read_csv(path+'List of Websites.csv')
streetsuffixdata=pd.read_csv(path+'Street Suffix.csv')
streetsuffixdata.columns=['strtsufx']
stateabrvdata=pd.read_csv(path+'State Abriviation.csv')
stateabrvdata.columns=['stateabrv']

url=listofwebsitesdata.url.values
outlet=listofwebsitesdata.Outlet.values
strtsufx=streetsuffixdata.strtsufx.values
strtsufx=[' '+x for x in strtsufx]
stateabrv=stateabrvdata.stateabrv.values
stateabrv=[' '+x for x in stateabrv]


# url=['http://www.bearclawcoffee.com/locations/','http://ontapgrille.com/','http://www.windmillhotdogs.com/home']
# outlet=['a','b','c']
# masterdf=pd.DataFrame(columns=['outletnum','outleturl',"Address"])

for x,y in zip(url,outlet):
    text=gettext(x)
    adrs1=findadrs1(text,strtsufx,stateabrv)
    adrs2=findadrs2(text,adrs1,stateabrv)
    posadrs=findadrs3(text,adrs1,stateabrv)
    adrs=adrs1+adrs2
    adrs.sort()
    adrs=dedup(adrs)
    output=[[y,x,i] for i in adrs]
    output+=[[y,x,'NEED CHECK: '+i] for i in posadrs]
    output=pd.DataFrame(output,columns=['outletnum','outleturl',"Address"])
    masterdf=masterdf.append(output,ignore_index=True)


print(masterdf.head(50))
